
# E-commerce Sales Analysis (Internship Project)

**Objective:** Analyze sales trends, customer behavior, product performance, and payment preferences to generate actionable business insights.

**Dataset:** `data/ecommerce_sales.csv` (synthetic: 1,000 rows; 2023-01-01 onward)

**Key Questions**
1. How do sales trend over time (daily, monthly)?  
2. Which products and categories perform best?  
3. What are the regional and payment method patterns?  
4. What recommendations can improve revenue?

> Run cells in order. This notebook is GitHub-ready.


In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Settings
pd.set_option('display.max_columns', None)

# Paths
DATA_PATH = '../data/ecommerce_sales.csv'
OUTPUT_DIR = '../output'


In [ ]:

df = pd.read_csv(DATA_PATH, parse_dates=['OrderDate'])
df.head()


In [ ]:

df.info()


In [ ]:

# Basic cleaning
df = df.drop_duplicates(subset=['OrderID'])
# Ensure types
df['Category'] = df['Category'].astype('category')
df['Product'] = df['Product'].astype('category')
df['Region'] = df['Region'].astype('category')
df['PaymentMethod'] = df['PaymentMethod'].astype('category')

# Derive date parts
df['Year'] = df['OrderDate'].dt.year
df['Month'] = df['OrderDate'].dt.to_period('M').dt.to_timestamp()
df['Week'] = df['OrderDate'].dt.to_period('W').dt.start_time

df.head()


In [ ]:

monthly_sales = df.groupby('Month', as_index=False)['TotalAmount'].sum()
category_sales = df.groupby('Category', as_index=False)['TotalAmount'].sum().sort_values('TotalAmount', ascending=False)
product_sales = df.groupby('Product', as_index=False)['TotalAmount'].sum().sort_values('TotalAmount', ascending=False)
region_sales = df.groupby('Region', as_index=False)['TotalAmount'].sum().sort_values('TotalAmount', ascending=False)
payment_counts = df['PaymentMethod'].value_counts().rename_axis('PaymentMethod').reset_index(name='Count')

monthly_sales.head(), category_sales.head(), product_sales.head(), region_sales.head(), payment_counts.head()


In [ ]:

import os
os.makedirs(OUTPUT_DIR, exist_ok=True)

# 1) Monthly sales trend
plt.figure()
plt.plot(monthly_sales['Month'], monthly_sales['TotalAmount'])
plt.title('Monthly Sales Trend')
plt.xlabel('Month')
plt.ylabel('Total Sales')
plt.tight_layout()
plt.savefig(f"{OUTPUT_DIR}/monthly_sales_trend.png")
plt.close()

# 2) Sales by Category
plt.figure()
plt.bar(category_sales['Category'].astype(str), category_sales['TotalAmount'])
plt.title('Sales by Category')
plt.xlabel('Category')
plt.ylabel('Total Sales')
plt.xticks(rotation=30, ha='right')
plt.tight_layout()
plt.savefig(f"{OUTPUT_DIR}/sales_by_category.png")
plt.close()

# 3) Top 10 Products
top10_products = product_sales.head(10)
plt.figure()
plt.bar(top10_products['Product'].astype(str), top10_products['TotalAmount'])
plt.title('Top 10 Products by Sales')
plt.xlabel('Product')
plt.ylabel('Total Sales')
plt.xticks(rotation=30, ha='right')
plt.tight_layout()
plt.savefig(f"{OUTPUT_DIR}/top10_products.png")
plt.close()

# 4) Orders by Payment Method
plt.figure()
plt.bar(payment_counts['PaymentMethod'].astype(str), payment_counts['Count'])
plt.title('Orders by Payment Method')
plt.xlabel('Payment Method')
plt.ylabel('Number of Orders')
plt.xticks(rotation=30, ha='right')
plt.tight_layout()
plt.savefig(f"{OUTPUT_DIR}/orders_by_payment.png")
plt.close()

'Plots saved to output/'


In [ ]:

# Key metrics & insights
total_revenue = df['TotalAmount'].sum()
avg_order_value = df['TotalAmount'].mean()
orders = len(df)
best_month = monthly_sales.loc[monthly_sales['TotalAmount'].idxmax(), 'Month']
best_category = category_sales.iloc[0]['Category']
best_region = region_sales.iloc[0]['Region']
top_product = product_sales.iloc[0]['Product']

summary = {
    'Total Revenue': float(round(total_revenue, 2)),
    'Average Order Value': float(round(avg_order_value, 2)),
    'Total Orders': int(orders),
    'Best Month (by sales)': str(best_month.date()),
    'Top Category': str(best_category),
    'Top Region': str(best_region),
    'Top Product': str(top_product)
}
summary


In [ ]:

# Save summary tables
monthly_sales.to_csv(f"{OUTPUT_DIR}/monthly_sales.csv", index=False)
category_sales.to_csv(f"{OUTPUT_DIR}/category_sales.csv", index=False)
product_sales.to_csv(f"{OUTPUT_DIR}/product_sales.csv", index=False)
region_sales.to_csv(f"{OUTPUT_DIR}/region_sales.csv", index=False)
payment_counts.to_csv(f"{OUTPUT_DIR}/orders_by_payment.csv", index=False)
'CSV exports saved to output/'
